In [ ]:
import ast

# Visitor to detect unreachable code (like code after a `return` statement)
class UnreachableCodeVisitor(ast.NodeVisitor):
    def __init__(self):
        self.unreachable_nodes = []     # Nodes detected as unreachable
        self.scope_stack = []           # Stack to keep track of current scope

    def visit_FunctionDef(self, node):
        self.scope_stack.append(node.name)
        self.generic_visit(node)
        self.scope_stack.pop()

    def visit_Return(self, node):
        self._detect_unreachable_code(node)

    def visit_Break(self, node):
        self._detect_unreachable_code(node)

    def visit_Continue(self, node):
        self._detect_unreachable_code(node)

    def visit_If(self, node):
        self.generic_visit(node)
        # Check if there's unreachable code after an "if" statement block ends
        if isinstance(node.orelse, list) and not node.orelse:
            self._detect_unreachable_code(node)

    def _detect_unreachable_code(self, node):
        if self.scope_stack:
            self.unreachable_nodes.append((self.scope_stack[-1], node.lineno))

# Checker to report unreachable code violations
class UnreachableCodeChecker:
    def __init__(self):
        self.violations = set()

    def check_unreachable(self, unreachable_nodes):
        # Report each unreachable code instance
        for scope, lineno in unreachable_nodes:
            self.violations.add(f"Violation: Unreachable code detected in '{scope}' at line {lineno}.")

# Sample code to test UnreachableCodeVisitor and UnreachableCodeChecker
code = """
def exampleFunction():
    x = 5
    return x
    print('This is unreachable')
    y = 10
"""
 #
# Parse the code
tree = ast.parse(code)

# Run UnreachableCodeVisitor
unreachable_visitor = UnreachableCodeVisitor()
unreachable_visitor.visit(tree)

# Run UnreachableCodeChecker
unreachable_checker = UnreachableCodeChecker()
unreachable_checker.check_unreachable(unreachable_visitor.unreachable_nodes)

# Print the unreachable code violations
print("Unreachable code violations:", unreachable_checker.violations)


Unreachable code violations: {"Violation: Unreachable code detected in 'exampleFunction' at line 4."}
